step - mapeia uma unidade de tempo no mundo real. Neste caso, 1 passo equivale a 1 hora. Total de passos 744 (simulação de 30 dias).

tipo - CASH-IN, CASH-OUT, DÉBITO, PAGAMENTO e TRANSFERÊNCIA.

montante - valor da transação em moeda local.

nameOrig – cliente que iniciou a transação

oldbalanceOrg - saldo inicial antes da transação

newbalanceOrig - novo saldo após a transação

nameDest - cliente que é o destinatário da transação

oldbalanceDest - destinatário do saldo inicial antes da transação. Observe que não há informações para clientes que iniciam com M (Comerciantes).

newbalanceDest - novo destinatário do saldo após a transação. Observe que não há informações para clientes que iniciam com M (Comerciantes).

isFraud - São as transações realizadas pelos agentes fraudulentos dentro da simulação. Neste conjunto de dados específico, o comportamento fraudulento dos agentes visa lucrar assumindo o controle das contas dos clientes e tentando esvaziar os fundos transferindo para outra conta e depois sacando do sistema.

isFlaggedFraud – O modelo de negócios visa controlar transferências massivas de uma conta para outra e sinalizar tentativas ilegais. Uma tentativa ilegal neste conjunto de dados é uma tentativa de transferir mais de 200.000 em uma única transação.

In [1]:
atalho para pegar oberservao da funcao - shift + tab

SyntaxError: invalid syntax (1308736298.py, line 1)

In [5]:
# Basic Tools
import numpy as np
import pandas as pd
from datetime import datetime as dt

import inflection
import re


# File/ OS Tools
import os
import sys
from watermark import watermark

In [6]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    accent = lambda x: inflection.transliterate(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_old = list(map(accent, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

In [7]:
data_raw = pd.read_csv('../data/raw/data_raw.csv',low_memory=False)
data_raw = rename_columns(data_raw)


In [34]:
data = data_raw.copy()

# Missing Data

In [35]:
data.isna().sum()

step                0
type                0
amount              0
name_orig           0
oldbalance_org      0
newbalance_orig     0
name_dest           0
oldbalance_dest     0
newbalance_dest     0
is_fraud            0
is_flagged_fraud    0
dtype: int64

# TIPO DE DADOS 

In [36]:
data.dtypes

step                  int64
type                 object
amount              float64
name_orig            object
oldbalance_org      float64
newbalance_orig     float64
name_dest            object
oldbalance_dest     float64
newbalance_dest     float64
is_fraud              int64
is_flagged_fraud      int64
dtype: object

In [37]:
data.head()

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [38]:
data['type'].value_counts(normalize=True)

type
CASH_OUT    0.351663
PAYMENT     0.338146
CASH_IN     0.219923
TRANSFER    0.083756
DEBIT       0.006512
Name: proportion, dtype: float64

In [39]:
data['is_fraud'].value_counts(normalize=True)

is_fraud
0    0.998709
1    0.001291
Name: proportion, dtype: float64

In [40]:
data['is_fraud'].value_counts(normalize=True,ascending=True)

is_fraud
1    0.001291
0    0.998709
Name: proportion, dtype: float64

In [41]:
data.loc[data['is_fraud'] == 1].sample(5,ignore_index=True)

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud
0,528,CASH_OUT,311234.46,C1795452410,311234.46,0.00,C1484612212,978976.44,1290210.90,1,0
1,225,CASH_OUT,84703.87,C1976888667,84703.87,0.00,C239391971,773479.20,858183.06,1,0
2,710,TRANSFER,10000000.00,C1004068843,27418265.31,17418265.31,C2116384638,0.00,0.00,1,0
3,533,CASH_OUT,3439121.37,C1650955113,3439121.37,0.00,C2091086284,67310.12,3506431.49,1,0
4,388,TRANSFER,20393.12,C1860777745,20393.12,0.00,C2094058620,0.00,0.00,1,0


In [42]:
(data['is_fraud'] == 1).sum()

8213

In [43]:
num_attributes = data.select_dtypes( include=['int64', 'float64'] )
cat_attributes = data.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

In [44]:
# Central Tendency - mean, meadina 
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T 
d2 = pd.DataFrame( num_attributes.apply( min ) ).T 
d3 = pd.DataFrame( num_attributes.apply( max ) ).T 
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T 
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T 
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T 

# concatenar
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,step,1.0,7.430000e+02,7.420000e+02,2.433972e+02,239.000,1.423320e+02,0.375177,0.329071
1,amount,0.0,9.244552e+07,9.244552e+07,1.798619e+05,74871.940,6.038582e+05,30.993949,1797.956705
2,oldbalance_org,0.0,5.958504e+07,5.958504e+07,8.338831e+05,14208.000,2.888242e+06,5.249136,32.964879
3,newbalance_orig,0.0,4.958504e+07,4.958504e+07,8.551137e+05,0.000,2.924048e+06,5.176884,32.066985
4,oldbalance_dest,0.0,3.560159e+08,3.560159e+08,1.100702e+06,132705.665,3.399180e+06,19.921758,948.674125
5,newbalance_dest,0.0,3.561793e+08,3.561793e+08,1.224996e+06,214661.440,3.674129e+06,19.352302,862.156508
6,is_fraud,0.0,1.000000e+00,1.000000e+00,1.290820e-03,0.000,3.590479e-02,27.779538,769.702982
7,is_flagged_fraud,0.0,1.000000e+00,1.000000e+00,2.514687e-06,0.000,1.585775e-03,630.603629,397659.062500


In [45]:
data['name_orig'] = data['name_orig'].astype(str)  # Convert integers to strings
data['length__name_orig'] = data['name_orig'].str.len()

In [46]:
data['name_dest'] = data['name_dest'].astype(str)  # Convert integers to strings
data['length__name_dest'] = data['name_dest'].str.len()


In [47]:
def deixar_1_letra(df, column_name):
    """ deixar a primeira letra do codigo do cliente origem e destino para tornar codigo inteiro"""
    data['type_dest'] = data[column_name].str[0:1]



In [48]:
deixar_1_letra(data, 'name_dest')

In [49]:
def remove_1_letra(df, column_name):
    """ retirando a primeira letra do codigo do cliente origem e destino para tornar codigo inteiro"""
    df[column_name] = df[column_name].str[1:]




In [50]:
remove_1_letra(data, 'name_orig')
remove_1_letra(data, 'name_dest')

In [51]:
data.head()

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud,LENGTH_name_orig,LENGTH_name_dest,type_dest
0,1,PAYMENT,9839.64,1231006815,170136.0,160296.36,1979787155,0.0,0.0,0,0,11,11,M
1,1,PAYMENT,1864.28,1666544295,21249.0,19384.72,2044282225,0.0,0.0,0,0,11,11,M
2,1,TRANSFER,181.00,1305486145,181.0,0.00,553264065,0.0,0.0,1,0,11,10,C
3,1,CASH_OUT,181.00,840083671,181.0,0.00,38997010,21182.0,0.0,1,0,10,9,C
4,1,PAYMENT,11668.14,2048537720,41554.0,29885.86,1230701703,0.0,0.0,0,0,11,11,M


In [52]:
# Defina a função de conversão
def converter_para_dia_hora(dataframe, step_column_name):
    """ Criando duas novas colunas de dia e hora levando em consideração o array que temos de 1 até 744, por exemplo
    o step 1 será dia 1 e hora 0  e quando for 25 será dias 2 e hora 0"""
    if step_column_name not in dataframe.columns:
        return "Coluna 'step' não encontrada no DataFrame"
    
    dataframe['Dia'] = (dataframe[step_column_name] - 1) // 24 + 1  # Cria a coluna 'Dia'
    dataframe['Hora'] = (dataframe[step_column_name] - 1) % 24       # Cria a coluna 'Hora'
    
    return dataframe


converter_para_dia_hora(data, 'step')

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud,LENGTH_name_orig,LENGTH_name_dest,type_dest,Dia,Hora
0,1,PAYMENT,9839.64,1231006815,170136.00,160296.36,1979787155,0.00,0.00,0,0,11,11,M,1,0
1,1,PAYMENT,1864.28,1666544295,21249.00,19384.72,2044282225,0.00,0.00,0,0,11,11,M,1,0
2,1,TRANSFER,181.00,1305486145,181.00,0.00,553264065,0.00,0.00,1,0,11,10,C,1,0
3,1,CASH_OUT,181.00,840083671,181.00,0.00,38997010,21182.00,0.00,1,0,10,9,C,1,0
4,1,PAYMENT,11668.14,2048537720,41554.00,29885.86,1230701703,0.00,0.00,0,0,11,11,M,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,786484425,339682.13,0.00,776919290,0.00,339682.13,1,0,10,10,C,31,22
6362616,743,TRANSFER,6311409.28,1529008245,6311409.28,0.00,1881841831,0.00,0.00,1,0,11,11,C,31,22
6362617,743,CASH_OUT,6311409.28,1162922333,6311409.28,0.00,1365125890,68488.84,6379898.11,1,0,11,11,C,31,22
6362618,743,TRANSFER,850002.52,1685995037,850002.52,0.00,2080388513,0.00,0.00,1,0,11,11,C,31,22


In [53]:
data.head()

,step,type,amount,name_orig,oldbalance_org,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud,LENGTH_name_orig,LENGTH_name_dest,type_dest,Dia,Hora
0,1,PAYMENT,9839.64,1231006815,170136.0,160296.36,1979787155,0.0,0.0,0,0,11,11,M,1,0
1,1,PAYMENT,1864.28,1666544295,21249.0,19384.72,2044282225,0.0,0.0,0,0,11,11,M,1,0
2,1,TRANSFER,181.00,1305486145,181.0,0.00,553264065,0.0,0.0,1,0,11,10,C,1,0
3,1,CASH_OUT,181.00,840083671,181.0,0.00,38997010,21182.0,0.0,1,0,10,9,C,1,0
4,1,PAYMENT,11668.14,2048537720,41554.0,29885.86,1230701703,0.0,0.0,0,0,11,11,M,1,0


In [54]:
tolerancia = 1e-6  # Defina a tolerância conforme necessário

# Calcule a diferença entre as colunas
diferenca = data['amount'] - (data['oldbalance_org'] - data['newbalance_orig'])

# Substitua valores muito próximos de zero por zero usando isclose()
data['origin_missing'] = np.where(np.isclose(diferenca, 0, atol=tolerancia), 0, diferenca)

In [55]:
tolerancia = 1e-6  # Defina a tolerância conforme necessário

# Calcule a diferença entre as colunas
diferenca = data['amount'] - (data['oldbalance_dest'] - data['newbalance_dest'])

# Substitua valores muito próximos de zero por zero usando isclose()
data['dest_missing'] = np.where(np.isclose(diferenca, 0, atol=tolerancia), 0, diferenca)

In [56]:
data['dest_missing'].fillna(0, inplace = True)

In [57]:
data = rename_columns(data)

In [59]:
data['name_dest'] = data['name_dest'].astype('int64')
data['name_orig'] = data['name_orig'].astype('int64')

In [62]:
data.dtypes

step                  int64
type                 object
amount              float64
name_orig             int64
oldbalance_org      float64
newbalance_orig     float64
name_dest             int64
oldbalance_dest     float64
newbalance_dest     float64
is_fraud              int64
is_flagged_fraud      int64
length_name_orig      int64
length_name_dest      int64
type_dest            object
dia                   int64
hora                  int64
origin_missing      float64
dest_missing        float64
dtype: object

In [65]:
data.isna().sum()

step                0
type                0
amount              0
name_orig           0
oldbalance_org      0
newbalance_orig     0
name_dest           0
oldbalance_dest     0
newbalance_dest     0
is_fraud            0
is_flagged_fraud    0
length_name_orig    0
length_name_dest    0
type_dest           0
dia                 0
hora                0
origin_missing      0
dest_missing        0
dtype: int64

In [66]:
data.columns

Index(['step', 'type', 'amount', 'name_orig', 'oldbalance_org',
       'newbalance_orig', 'name_dest', 'oldbalance_dest', 'newbalance_dest',
       'is_fraud', 'is_flagged_fraud', 'length_name_orig', 'length_name_dest',
       'type_dest', 'dia', 'hora', 'origin_missing', 'dest_missing'],
      dtype='object')

In [64]:
data.to_csv("../data/processed/data_processed.csv", index=False)